In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input, UpSampling2D, Conv2D, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
(X_train, _), (_, _) = mnist.load_data()

X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

In [3]:
noise_dim = 100

In [4]:
generator = Sequential([
    Dense(128 * 7 * 7, input_dim=noise_dim),
    Reshape((7, 7, 128)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    UpSampling2D(),
    Conv2D(64, (5, 5), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    UpSampling2D(),
    Conv2D(1, (5, 5), padding='same', activation='tanh')
])

In [5]:
discriminator = Sequential([
    Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),
    Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [6]:
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

discriminator.trainable = False

gan_input = Input(shape=(noise_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [7]:
def train_gan(epochs=1, batch_size=128):
    batch_count = X_train.shape[0] // batch_size

    for e in range(epochs):
        for _ in range(batch_count):
            noise = np.random.normal(0, 1, size=[batch_size, noise_dim])
            generated_images = generator.predict(noise)
            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

            X = np.concatenate([image_batch, generated_images])
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # Etiquetas suavizadas

            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            noise = np.random.normal(0, 1, size=[batch_size, noise_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        print(f'Época {e}:\tD Loss: {d_loss[0]} | Accuracy: {100 * d_loss[1]}%\tG Loss: {g_loss}')

In [8]:
train_gan(epochs=100, batch_size=128)

4/4 [==============================] - 0s 11ms/step
Época 0:	D Loss: 0.5734015107154846 | Accuracy: 46.484375%	G Loss: 0.6841179132461548
4/4 [==============================] - 0s 11ms/step
Época 1:	D Loss: 0.6814560890197754 | Accuracy: 42.1875%	G Loss: 0.9010000228881836
4/4 [==============================] - 0s 11ms/step
Época 2:	D Loss: 0.6797931790351868 | Accuracy: 38.28125%	G Loss: 0.7653538584709167
4/4 [==============================] - 0s 11ms/step
Época 3:	D Loss: 0.6673572063446045 | Accuracy: 46.484375%	G Loss: 0.9302223920822144
4/4 [==============================] - 0s 10ms/step
Época 4:	D Loss: 0.6424105763435364 | Accuracy: 41.796875%	G Loss: 0.992499053478241
4/4 [==============================] - 0s 10ms/step
Época 5:	D Loss: 0.622289776802063 | Accuracy: 44.921875%	G Loss: 0.8898509740829468
4/4 [==============================] - 0s 12ms/step
Época 6:	D Loss: 0.5900291204452515 | Accuracy: 40.625%	G Loss: 0.938398003578186
4/4 [==============================] - 0s 9

In [ ]:
noise = np.random.normal(0, 1, size=[16, noise_dim])
generated_images = generator.predict(noise)

plt.figure(figsize=(4, 4))
for i in range(generated_images.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')
plt.tight_layout()
plt.show()